# 아이펠톤에서 사용할 데이터 생성하는 노트북

### 발생한 이슈
- 같은 텍스트를 넣었는데, openai api에서 포함해야 하는 텍스트를 제외하는 일이 발생함 -> 문장유사도를 판단하여, 유사한 문장을 포함하지 않으면 제외하는 방식 사용

### 코드 흐름
- 데이터를 한줄씩 가져옴
- 포함해야 하는 컬럼의 데이터를 openai api 에 넣어 문장을 생성함
- 포함해야 하는 컬럼의 데이터를 문장별로 나눔
- 생성된 문장 내에 '포함해야 하는 문장'들이 존재하는지 문장유사도를 사용하여 확인
- 문장유사도 비율이 낮으면 중단하고 저장, 원인 확인

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import nltk
from difflib import SequenceMatcher
import winsound
import ftfy
import re
import string
import glob

# NLTK 패키지에서 Punkt tokenizer를 다운로드 (한 번만 실행)
# nltk.download('punkt')

In [4]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ.get("API_KEY")

client = OpenAI()

In [3]:
# 문장 부호를 제거한 후, 소문자로 변환하고 단어로 분리하는 함수
def clean_and_split(sentence):
    # 문장 부호 제거 (string.punctuation을 사용하여 기본적인 문장 부호 제거)
    cleaned_sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    
    # 소문자로 변환하고 단어로 분리
    words = cleaned_sentence.lower().split()
    
    return words

# 두 문장 간의 포함 비율을 계산하는 함수
# base 문장이 story에 포함되어 있는 경우 
def partial_inclusion_ratio(base_sentence, sentence):
    # base_sentence와 sentence의 길이를 기준으로 공통 문자를 확인
    base_words = list(clean_and_split(base_sentence))
    sentence_words = list(clean_and_split(sentence))

    # 공통 문자를 찾기 위해 base_sentence의 문자들이 sentence에 얼마나 포함되는지 확인
    common_words = [word for word in base_words if word in sentence_words]
    
    # 포함된 문자 비율을 계산
    if len(base_words) == 0:
        inclusion_ratio = 0
    else:
        inclusion_ratio = len(common_words) / len(base_words)
    
    return inclusion_ratio

# 문장유사도 비교
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# '포함해야 하는 문장들' 내의 1개의 문장과 '만들어진 장문' 내 문장들을 비교하여, 가장 유사한 문장을 1개 반환 
def find_similar_sentences(base_sentence, long_text, threshold=0):
    # 긴 텍스트를 문장으로 분리
    sentences = nltk.sent_tokenize(long_text)
    
    # 유사한 문장 추출
    base = ""
    long = ""
    similarity_final = 0
    include_rate = 0.7
    for sentence in sentences:
        # 포함 비율 계산
        inclusion_ratio = partial_inclusion_ratio(base_sentence, sentence)
        
        # base 문장이 stroy 문장에 부분 포함관계에 있는지 확인
        if inclusion_ratio >= include_rate:
            similarity = inclusion_ratio*100  # 유사도를 1 이상으로 설정
        else:
            similarity = similar(base_sentence, sentence)  # 포함되지 않으면 유사도 계산
        
        # 유사도 소수점 셋째 자리까지 반올림
        similarity = round(similarity, 3)

        if similarity >= threshold and similarity > similarity_final:
            base = base_sentence
            long = sentence
            similarity_final = similarity
    return base, long, similarity_final

In [4]:
# api함수 
def get_chatgpt_response_c2d2(input_01, input_02):
    # OpenAI API를 통해 ChatGPT에게 한국어로 자연스럽게 다듬어 달라고 요청
    response = client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4o-mini",
        # model ="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a patient receiving psychological counseling. who can speak english only."},
            {"role": "user", 
             "content": f"다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"}
        ]
    )
    # return response['choices'][0]['message']['content']
    return response

### 아래 데이터를 생성함
- annotated data (캐글의 심리학자-환자 대화 셋)의 전체 장문, 상담 받는 사람의 생각, 해당 생각의 인지왜곡 type 과 포멧을 맞춤
- (중국어 -> 영어로 변역된) C2D2 데이터셋의 시나리오+thought (인지 왜곡된 생각)을 맞춰 장문을 만들어냄
- 생성하며, 문장유사도가 낮은 문장이 포함된다면 생성을 멈출 것 -> 전체 데이터가 잘 못 생성되는 경우가 있음



- 선행연구 C2d2 dataset: A resource for the cognitive distortion analysis and its impact on mental health. 에서 사용된 데이터셋임

In [14]:
raw = pd.read_csv("raw_data/Final_Modified_C2D20911.csv")
raw.head()

,Num,Scenario,Thought,Label
0,1,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?,Overgeneralization
1,2,"Recently, I feel dizzy sometimes when I stand ...",I'm so dizzy. Am I sick? I should probably go ...,No Distortion
2,3,"I'm walking down the street and feel hungry, b...","I'm tired and there's no place to rest, I'm hu...",Overgeneralization
3,4,"Work has been busy lately, but I have caught a...",Why did I catch a cold at this time? I feel so...,No Distortion
4,5,My mom and I are discussing future plans. She ...,"Mom is trying to control my life again, wantin...",Fortune-telling


In [25]:
raw.iloc[1024]

Num                                                                1025
Scenario              I discover that I am not my parents' biologica...
Thought               I’m sorry, I’m sorry. I’m obviously not my par...
Label                                                   Personalization
Scenario_Sentences    [I discover that I am not my parents' biologic...
Thought_Sentences     [I’m sorry., I’m sorry., I’m obviously not my ...
all_Sentences         [I discover that I am not my parents' biologic...
Name: 1024, dtype: object

In [15]:
## 번역 잘못된 부분 처리
# 'Ir'을 'my'로 변경
raw['Scenario'] = raw['Scenario'].str.replace(' Ir ', ' my ', case=False)
raw['Scenario'] = raw['Scenario'].str.replace(' i, ', ' me, ', case=False)
raw['Scenario'] = raw['Scenario'].str.replace('I are ', 'I am ', case=False)
raw['Scenario'] = raw['Scenario'].str.replace('i are ', 'i am ', case=False)

raw['Thought'] = raw['Thought'].str.replace(' Ir ', ' my ', case=False)
raw['Thought'] = raw['Thought'].str.replace(' i, ', ' me, ', case=False)
raw['Thought'] = raw['Thought'].str.replace('I are ', 'I am ', case=False)
raw['Thought'] = raw['Thought'].str.replace('i are ', 'i am ', case=False)


# 포함해야 하는 데이터들을 문장 단위로 쪼갬
# , 가 있는 문장을 생성할떄 .으로 바뀌며 두개로 나뉠 수 있으므로 변경함
raw['Scenario_Sentences'] = raw['Scenario'].str.replace(', ', '. ', case=False)
raw['Thought_Sentences'] = raw['Thought'].str.replace(', ', '. ', case=False)

raw['Scenario_Sentences'] = raw['Scenario_Sentences'].apply(lambda x: nltk.sent_tokenize(x))
raw['Thought_Sentences'] = raw['Thought_Sentences'].apply(lambda x: nltk.sent_tokenize(x))

raw['all_Sentences'] = raw['Scenario_Sentences'] + raw['Thought_Sentences']
raw['all_Sentences']

0       [I'm an introverted person., and I've just arr...
1       [Recently., I feel dizzy sometimes when I stan...
2       [I'm walking down the street and feel hungry.,...
3       [Work has been busy lately., but I have caught...
4       [My mom and I am discussing future plans., She...
                              ...                        
7498    [I want to be tall., rich and handsome., but I...
7499    [I feel that I have poor expressive skills and...
7500    [Frequent overtime work makes I feel too uncom...
7501    [I feel that I have poor expressive skills and...
7502    [I woke up in the morning and wanted to sleep ...
Name: all_Sentences, Length: 7503, dtype: object

### 한개만 실행해봄

In [8]:
# 한개만 실행해봄
index_num = 0

refined_response = get_chatgpt_response_c2d2(raw["Scenario"][index_num], raw["Thought"][index_num])

# 결과 출력
print(refined_response.choices[0].message.content)

I'm an introverted person, and I've just arrived in a new environment where the people around I are mostly unfamiliar. This change has made me feel quite anxious and unsure about how to proceed. I find myself worrying whether I will be able to connect with anyone here. Are the people in this environment unfriendly? Sometimes, I think it might be my own fears holding me back from trying.


In [13]:
# 결과 저장할 데이터프레임 생성
result_df = pd.DataFrame()
    
base_list = []
long_list = []
similarity_list = []

long_text = refined_response.choices[0].message.content

for base_text in raw.iloc[index_num]['all_Sentences']:
    # print(base_text)
    base, long, similarity = find_similar_sentences(base_text, long_text)

    base_list.append(base)
    long_list.append(long)
    similarity_list.append(similarity)

# 유사도가 있는 경우 평균 계산, 없으면 0으로 설정
if similarity_list:
    average_similarity = sum(similarity_list) / len(similarity_list)
else:
    average_similarity = 0
    
# 데이터프레임으로 만들어서 비교
result = pd.DataFrame({
    'base': [base_list],
    'long': [long_list],
    'similarity': [similarity_list],
    'average_similarity': average_similarity
})

result_df = pd.concat([result_df, result], ignore_index=True)

# 저장
result_df.to_csv("data/re.csv", index=False)
# result_df.to_csv(file_path_list, mode='a', header=False, index=False)  # 이후 100개씩 추가


### 전체 데이터 반복 실행
코드 구조
- 문장 생성
- cleaning (chatgpt가 가끔 의견을 덧붙힐 때가 있음 - 그걸 제거함)
- 유사도 비교, 데이터 프레임으로 만들어서 합침
- 유사도가 낮은 문장이 나오거나, 루프가 끝나면 저장

In [130]:
start_num = 7405 # 쓸꺼면 엑셀 숫자 다음꺼

raw_filtered = raw.iloc[start_num:]
print(len(raw_filtered))
print(raw_filtered.head(1))

98
       Num                                           Scenario  \
7405  7406  Suddenly I'm walking outside and realize my cl...   

                                                Thought          Label  \
7405  It's over. This is so embarrassing. I wanted t...  No Distortion   

                                     Scenario_Sentences  \
7405  [Suddenly I'm walking outside and realize my c...   

                                      Thought_Sentences  \
7405  [It's over., This is so embarrassing., I wante...   

                                          all_Sentences  
7405  [Suddenly I'm walking outside and realize my c...  


In [131]:
# 반복문
result_df = pd.DataFrame() # 결과 저장할 데이터프레임 생성
end_num = len(raw_filtered)

for i, row in raw_filtered.iterrows():
    # print(row)
    
    # 문장 생성
    refined_response = get_chatgpt_response_c2d2(row["Scenario"], row["Thought"])

    # 생성된 문장 확인 
    gan_sentence = refined_response.choices[0].message.content
    # print(gan_sentence)

    # 문장 필터링
    # story = clean_refined_thought(gan_sentence)
    # print(story)

    # 유사도 비교
    base_list = []
    long_list = []
    similarity_list = []

    for base_text in row['all_Sentences']:
        # print(base_text)
        base, long, similarity = find_similar_sentences(base_text, gan_sentence)

        base_list.append(base)
        long_list.append(long)
        similarity_list.append(similarity)

    # 유사도가 있는 경우 평균 계산, 없으면 0으로 설정
    if similarity_list:
        average_similarity = sum(similarity_list) / len(similarity_list)
    else:
        average_similarity = 0
        
    # 데이터프레임으로 만들어서 비교
    result = pd.DataFrame({
        'num' : row["Num"],
        'scenario' : row["Scenario"],
        'thought' : row["Thought"],
        'label' : row["Label"],
        'gen_sentence' : gan_sentence,
        # 'story' : story,
        'base': [base_list],
        'long': [long_list],
        'similarity': [similarity_list],
        'average_similarity': average_similarity
    })

    result_df = pd.concat([result_df, result], ignore_index=True)

    # 유사도가 일정 이하거나, 전체 데이터를 다 돌은 경우, 저장하고 루프 종료
    has_below = any(num < 0.6 for num in similarity_list)

    if has_below or i == end_num-1:
        print(f"유사도가 0.6 미만인 경우 발생: {row['Num']}")

        # 비프음 출력해서 알림
        # 주파수(Hz)와 지속 시간(ms) 설정
        frequency = 1000  # 주파수: 1000Hz
        duration = 5000    # 지속 시간: 500ms

        # 비프음 출력
        winsound.Beep(frequency, duration)
        
        # 정제 후 저장
        result_df['Cleaned'] = result_df['gen_sentence'].apply(clean_refined_thought)
        result_df.to_csv(f"data/gen_c2d2/c2d2_{start_num}_{i}.csv", index=False)
        
        break

In [135]:
result_df['Cleaned'] = result_df['gen_sentence'].apply(clean_refined_thought)
result_df.to_csv(f"data/gen_c2d2/c2d2_{start_num}_{i}.csv", index=False)

In [133]:
print(result_df.iloc[-1].gen_sentence)
print(result_df.iloc[-1].similarity)
print(result_df.iloc[-1].average_similarity)
print(result_df.iloc[-1].base)
print(result_df.iloc[-1].long)

Sure, I can help fill in the missing sentences while keeping the provided text unchanged:

I woke up in the morning and wanted to sleep a little longer, but when I woke up I found that I was late. I rushed to get ready and skipped breakfast to make up for lost time. Why didn't my roommates call me? They must have wanted to see me being scolded for being late. They must have been laughing at me secretly and wanted to embarrass me. I couldn't believe they would do that to me. It made me feel so alone and betrayed.
[100.0, 100.0, 100.0, 100.0, 100.0]
100.0
['I woke up in the morning and wanted to sleep a little longer.', 'but when I woke up I found that I was late.', "Why didn't my roommates call me?", 'They must have wanted to see me being scolded for being late.', 'They must have been laughing at me secretly and wanted to embarrass me.']
['Sure, I can help fill in the missing sentences while keeping the provided text unchanged:\n\nI woke up in the morning and wanted to sleep a little lo

### 데이터 합치기
- 따로따로 만들어진 데이터를 합침

In [56]:
# 폴더 경로 설정 (예: 'your_folder_path')
folder_path = 'data/gen_c2d2'

# 해당 폴더 내 모든 CSV 파일 경로 가져오기
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파일들을 저장할 리스트 생성
df_list = []

# 모든 CSV 파일을 불러와서 리스트에 추가
for file in all_files:
    df = pd.read_csv(file, encoding='ISO-8859-1')  # CSV 파일 읽기
    df_list.append(df)

# 리스트에 있는 모든 데이터프레임을 아래로 합치기
merged_df = pd.concat(df_list, ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
merged_df.to_csv('data/c2d2_after_gen_all.csv', index=False)

### 데이터 정제
- gpt 붙힌 사족 제거
- encoding문제로 깨져 보이는 것, 문법적으로 틀린 곳 교정 (눈에 보이는 것만 일단 처리)

In [16]:
# 데이터 불러오기
c2d2_gen_all = pd.read_csv("data/c2d2_after_gen_all.csv")
print(c2d2_gen_all.head(1))
print(len(c2d2_gen_all))

   num                                           scenario  \
0  1.0  I'm an introverted person, and I've just arriv...   

                                          thought               label  \
0  Are the people in this environment unfriendly?  Overgeneralization   

                                        gen_sentence  \
0  Sure, I can help with that:\n\n"I'm an introve...   

                                                base  \
0  ["I'm an introverted person.", "and I've just ...   

                                                long             similarity  \
0  ['Sure, I can help with that:\n\n"I\'m an intr...  [100.0, 100.0, 100.0]   

  average_similarity                                            Cleaned  ...  \
0              100.0  I'm an introverted person, and I've just arriv...  ...   

  Unnamed: 16 Unnamed: 17 Unnamed: 18 Unnamed: 19 Unnamed: 20  Unnamed: 21  \
0         NaN         NaN         NaN         NaN         NaN          NaN   

  Unnamed: 22 Unnamed: 23 U

In [17]:
# 잘못 합쳐진 데이터 제거
# c2d2_gen_all[c2d2_gen_all['Unnamed: 25'].notna()]
print(c2d2_gen_all.iloc[1728])
data = c2d2_gen_all.drop([1728])
data = data[['scenario', 'thought', 'label', 'gen_sentence', 'base', 'long', 'similarity', 'average_similarity', 'Cleaned']]
data.head(1)

num                                                               253.0
scenario              I find out that I didn?t save the PPT I worke...
thought               I worked on it for so long, but because I didn...
label                                               Emotional Reasoning
gen_sentence          Sure, I can help with that. Here?s the comple...
base                                      'I worked on it for so long.'
long                                  ""but because I didn't save it.""
similarity                                        'everything is gone.'
average_similarity                                 'I feel really bad.'
Cleaned                                    'So why did I work so hard?'
Unnamed: 10                                                   'Anyway.'
Unnamed: 11                                       'the time is wasted.'
Unnamed: 12            'so I might as well go to sleep and just do s...
Unnamed: 13           ['Here?s the completion of your diary ent

,scenario,thought,label,gen_sentence,base,long,similarity,average_similarity,Cleaned
0,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?,Overgeneralization,"Sure, I can help with that:\n\n""I'm an introve...","[""I'm an introverted person."", ""and I've just ...","['Sure, I can help with that:\n\n""I\'m an intr...","[100.0, 100.0, 100.0]",100.0,"I'm an introverted person, and I've just arriv..."


In [18]:
# 데이터가 없는 row 제거
data = data.dropna(subset=['gen_sentence'])
len(data)

7499

In [19]:
# encoding문제로 깨져 보이는 것 수정
# ftfy를 사용해 컬럼의 모든 텍스트의 인코딩 문제를 수정
data['gen_sentence'] = data['gen_sentence'].apply(ftfy.fix_text)

In [26]:
# 함수 정의: --- 이후의 텍스트를 추출하는 함수
def extract_reponse(text):
    # --- 사이에 텍스트가 있는지 확인
    pattern_01 = r'---\n(.*?)\n---'
    matches_01 = re.findall(pattern_01, text, re.DOTALL)

    # --- 이후 텍스트 추출
    pattern_02 = r'---\n(.*)'
    matches_02 = re.search(pattern_02, text, re.DOTALL)

    # 결과 반환
    if matches_01:
        result = matches_01[0]
    elif matches_02:
        result = matches_02.group(1)
    elif '\n' not in text: # 줄바꿈이 없는 경우
        result = text
    elif '\n' in text: # 줄바꿈이 있는 경우
        # ':' 뒤에 공백이 있으면 그 이전의 텍스트를 모두 제거함 (앞부분 사족은 대부분 이걸로 제거됨)-- 아이디어는 좋았는데, 다른 문장도 제거됨
        # pattern_03 = r'.*:\s+'
        # matches_03 = re.sub(pattern_03, '', text, flags=re.DOTALL)

        # 특정 문자열을 포함한 문장 제거를 위한 키워드 리스트
        remove_keywords = ["Sure,", "Here's the completion of the diary", "Here are your completed journal entries:",
                           "2. ",
                           "I hope this helps", "Would you like to", "Does this help", "you would like to add or edit", "I hope this fills in the missing parts",
                           ]
        # 텍스트를 문장 단위로 나누기
        sentences = text.split('\n')
        # 여러 문자열 중 하나라도 포함된 문장 제거
        filtered_sentences = [sentence for sentence in sentences if not any(keyword in sentence for keyword in remove_keywords)]
        # 문장들을 다시 결합
        result = ' '.join(filtered_sentences)
    else:
        result = None 

    # 잘못된 문법 정제
    if result is not None:
        result = result.strip().replace("?™", "'").replace("I are", "I am").replace("I were", "i am").replace("1. ", "").strip('"*')
    
    return result

# apply()로 컬럼의 모든 데이터에 함수 적용
data['Cleaned'] = data['gen_sentence'].apply(extract_reponse)

# 결과 출력
print(data['Cleaned'])


0       I'm an introverted person, and I've just arriv...
1       Recently, I feel dizzy sometimes when I stand ...
2       I'm walking down the street and feel hungry, b...
3       Work has been busy lately, but I have caught a...
4       My mom and I am discussing future plans. She m...
                              ...                        
7496    I want to be tall, rich and handsome, but I do...
7497    I feel that I have poor expressive skills and ...
7498    Certainly. Here's the continuation of your dia...
7499    I feel that I have poor expressive skills and ...
7500    I woke up in the morning and wanted to sleep a...
Name: Cleaned, Length: 7499, dtype: object


In [27]:
# 문장 수 세기
# 각 행의 텍스트에서 문장 수를 세는 함수 정의
def count_sentences(text):
    sentences = nltk.sent_tokenize(text)  # 문장을 토큰화
    return len(sentences)  # 문장의 개수 반환

# 'text_column'의 문장 수 계산하여 새로운 컬럼에 저장
data['sentence_count'] = data['Cleaned'].apply(count_sentences)

In [28]:
data['sentence_count'].value_counts()

sentence_count
5     2464
6     2127
4     1305
7      992
8      385
9      134
3       49
10      27
11       8
12       3
2        3
13       1
46       1
Name: count, dtype: int64

In [29]:
# sentence_count가 특정 숫자인 것 제거
data = data[data['sentence_count'] != 2]
data = data[data['sentence_count'] != 46]
data['sentence_count'].value_counts()

sentence_count
5     2464
6     2127
4     1305
7      992
8      385
9      134
3       49
10      27
11       8
12       3
13       1
Name: count, dtype: int64

In [30]:
i = 3
n = 0
# 검증
print(data[data['sentence_count']==i]['gen_sentence'].iloc[n])
print("----------------")
print(data[data['sentence_count']==i]['Cleaned'].iloc[n])

While walking on the road, a stray dog rushes up and bites me. I scream in pain and try to push the dog away, but it's too quick. I must have rabies now, my family and friends must be very scared of me, and I don't know what to do.
----------------
While walking on the road, a stray dog rushes up and bites me. I scream in pain and try to push the dog away, but it's too quick. I must have rabies now, my family and friends must be very scared of me, and I don't know what to do.


In [31]:
# 문장이 아예 없으면 안됨
print(data[data['Cleaned'].isna()])

Empty DataFrame
Columns: [scenario, thought, label, gen_sentence, base, long, similarity, average_similarity, Cleaned, sentence_count]
Index: []


In [32]:
# 확인
# print(data['Cleaned'].iloc[87])
print(data['gen_sentence'].iloc[87])

Sure, here is your completed diary entry including the provided sentences:

I am not a good-looking child and I feel a little inferior. It's hard to feel confident when all the advertisements and media focus on perfect faces and bodies. The world only cares about beautiful people: I was born ugly, I will be ugly all my life, and I don't think I will ever become more beautiful. Sometimes, it's tough to find value in myself outside of appearance, but I know I have other qualities that matter. It's a daily struggle to remind myself that beauty is not everything.


In [33]:
# 저장
data.to_csv("data/c2d2_after_gen_all_cleaned.csv", index=False)

In [3]:
# 문법 교정은 여러가지 라이브러리를 실행해보려 했는데, 설치가 제대로 되는 것이 없음
# grammerly 사이트를 웹 크롤링으로 수정하는게 좋을 듯 하며, 작업이 크므로 나중으로 넘김

ModuleNotFoundError: No module named 'gingerit.gingerit'

### annotated data 에 맞춰 데이터셋을 변경함
- 필요한 것은 아래와 같음
1. stroy
2. Distorted part
3. Distorted type

In [5]:
# 데이터 불러오기
dt = pd.read_csv("data/c2d2_after_gen_all_cleaned.csv")

In [6]:
# 인지왜곡 생각을 인코딩 문제 보정 후, 문장 단위로 나누기
dt['thought'] = dt['thought'].apply(ftfy.fix_text)

dt['Thought_Sentences'] = dt['thought'].str.replace(', ', '. ').str.replace("?™", "'").str.replace("I are", "I am").str.replace("I were", "i am")
dt['Thought_Sentences'] = dt['Thought_Sentences'].apply(lambda x: nltk.sent_tokenize(x))
dt['Thought_Sentences']

0        [Are the people in this environment unfriendly?]
1       [I'm so dizzy., Am I sick?, I should probably ...
2       [I'm tired and there's no place to rest., I'm ...
3       [Why did I catch a cold at this time?, I feel ...
4       [Mom is trying to control my life again., want...
                              ...                        
7490    [Anyway., no matter how hard you work., you wo...
7491    [I felt like I was being corrupted by somethin...
7492    [These overtime jobs made me feel the darkness...
7493    [As a newcomer., I had a feeling that I would ...
7494    [Why didn't my roommates call me?, They must h...
Name: Thought_Sentences, Length: 7495, dtype: object

In [7]:
# 최종 문장에서 유사도 높은 문장 찾아내서 합침 (인지왜곡 문장인 Distorted part가 됨)

result_df = pd.DataFrame() # 결과 저장할 데이터프레임 생성
rmv_num = 0

for i, row in dt.iterrows():
    # 유사도 비교
    base_list = []
    long_list = []
    similarity_list = []

    gan_sentence = row['Cleaned']
    # print(gan_sentence)

    for base_text in row['Thought_Sentences']:
        # print(base_text)
        base, long, similarity = find_similar_sentences(base_text, gan_sentence)

        base_list.append(base)
        long_list.append(long)
        similarity_list.append(similarity)

    # 너무 정확도가 낮은 문장이 있다면 제외 (약 20개 제외됨)
    has_below = any(num < 0.6 for num in similarity_list)
    if has_below:
        rmv_num += 1
        continue        
    
    # long 텍스트 내 중복 제거
    long_list = list(set(long_list))
    
    # 유사도가 있는 경우 평균 계산, 없으면 0으로 설정
    if similarity_list:
        average_similarity = sum(similarity_list) / len(similarity_list)
    else:
        average_similarity = 0
        
    # 데이터프레임으로 만들어서 비교
    result = pd.DataFrame({
        'story' : row['Cleaned'],
        'Distorted part': [long_list],
        'label' : row["label"],
        'Distorted_문장분리': [base_list],
        'Distorted_문장유사도': [similarity_list],
        'Distorted_문장유사도평균': average_similarity,
        '기존데이터_scenario' : row["scenario"],
        '기존데이터_thought' : row["thought"],
    })

    result_df = pd.concat([result_df, result], ignore_index=True)

print(rmv_num)

14


In [8]:
# 저장
result_df.to_csv(f"data/c2d2_0924_final.csv", index=False)

### annotated data 도 위 형식에 맞춰 약간 수정해줌

In [10]:
# 데이터 불러오기
dt = pd.read_csv("raw_data/Annotated_data.csv")
dt.head(1)

,Id_Number,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional)
0,4500,"Hello, I have a beautiful,smart,outgoing and a...",The voice are always fimilar (someone she know...,Personalization,NaN


In [15]:
import numpy as np
# 인지왜곡 생각을 인코딩 문제 보정 후, 문장 단위로 나누기

# Distorted part가 비어 있거나 NaN인 경우 그대로 두고, 그렇지 않으면 문장 단위로 나누기
dt['Distorted part'] = dt['Distorted part'].replace(np.nan, '')  # NaN을 빈 문자열로 치환

dt['Distorted_sen'] = dt['Distorted part'].apply(lambda x: nltk.sent_tokenize(x) if isinstance(x, str) and x.strip() else x)
dt['Distorted_sen']

0       [The voice are always fimilar (someone she kno...
1       [I feel trapped inside my disgusting self and ...
2                                                        
3                                                        
4       [I refused to go because I didn’t know if it w...
                              ...                        
2525                                                     
2526    [Now I am at university my peers around me all...
2527    [He claims he’s severely depressed and has out...
2528                                                     
2529                                                     
Name: Distorted_sen, Length: 2530, dtype: object

In [17]:
# 최종 문장에서 유사도 높은 문장 찾아내서 합침 (인지왜곡 문장인 Distorted part가 됨)

result_df = pd.DataFrame() # 결과 저장할 데이터프레임 생성
rmv_num = 0

for i, row in dt.iterrows():
    # 유사도 비교
    base_list = []
    long_list = []
    similarity_list = []

    gan_sentence = row['Patient Question']
    # print(gan_sentence)

    for base_text in row['Distorted_sen']:
        # print(base_text)
        base, long, similarity = find_similar_sentences(base_text, gan_sentence)

        base_list.append(base)
        long_list.append(long)
        similarity_list.append(similarity)

    # 너무 정확도가 낮은 문장이 있다면 제외 (약 20개 제외됨)
    has_below = any(num < 0.6 for num in similarity_list)
    # if has_below:
    #     rmv_num += 1
    #     continue 
    
    # long 텍스트 내 중복 제거
    long_list = list(set(long_list))
    
    # 유사도가 있는 경우 평균 계산, 없으면 0으로 설정
    if similarity_list:
        average_similarity = sum(similarity_list) / len(similarity_list)
    else:
        average_similarity = 0
        
    # 데이터프레임으로 만들어서 비교
    result = pd.DataFrame({
        'story' : row['Patient Question'],
        'Distorted part': [long_list],
        'label' : row["Dominant Distortion"],
        'Distorted_문장분리': [base_list],
        'Distorted_문장유사도': [similarity_list],
        'Distorted_문장유사도평균': average_similarity,
        '기존데이터_SecondaryDistortion' : row["Secondary Distortion (Optional)"],
    })

    result_df = pd.concat([result_df, result], ignore_index=True)

print(rmv_num)

0


In [18]:
# 저장
result_df.to_csv(f"data/annotated_change.csv", index=False)